In [1]:
#https://stackoverflow.com/questions/62165172/convert-dly-files-to-csv-using-python
import glob
import csv
import os
import pandas as pd
import datetime
import re

In [2]:
# ingest 
w_events = pd.read_csv('WeatherEvents_Jan2016-Dec2020.csv')


In [3]:
# examine 
w_events.head()


,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0


In [4]:
# check which event types seem worth including
print(f'weather types: {w_events["Type"].unique()}')

# i would say all are valid and should be kept in our dataset

weather types: ['Snow' 'Fog' 'Cold' 'Storm' 'Rain' 'Precipitation' 'Hail']


In [5]:
# check which event severities seem worth including
print(f'weather severities: {w_events["Severity"].unique()}')

# drop light, moderate, unk, other

weather severities: ['Light' 'Severe' 'Moderate' 'Heavy' 'UNK' 'Other']


In [6]:
# it looks like 'Severe' and 'Heavy' are most extreme
extreme_severities = ['Severe', 'Heavy']
extreme_w_events = w_events[w_events['Severity'].isin(extreme_severities)]
print(w_events.shape, extreme_w_events.shape)
print(f'extreme events represent about {round(100*(extreme_w_events.shape[0]/w_events.shape[0]),2)} percent of all events in the original data')

(6274206, 13) (1333526, 13)
extreme events represent about 21.25 percent of all events in the original data


In [7]:
extreme_w_events['wcoordinateID'] = list(zip(round(extreme_w_events['LocationLat'],2),round(extreme_w_events['LocationLng'],2)))
extreme_w_events.head()

C:\Users\Skurai\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,wcoordinateID
6,W-7,Fog,Severe,2016-01-09 12:54:00,2016-01-09 15:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,"(38.1, -106.17)"
8,W-9,Fog,Severe,2016-01-09 16:14:00,2016-01-09 16:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,"(38.1, -106.17)"
10,W-11,Cold,Severe,2016-01-09 16:54:00,2016-01-09 20:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,"(38.1, -106.17)"
11,W-12,Fog,Severe,2016-01-10 02:54:00,2016-01-10 04:14:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,"(38.1, -106.17)"
12,W-13,Fog,Severe,2016-01-10 09:34:00,2016-01-10 10:14:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,"(38.1, -106.17)"


In [8]:
w_events_date_start = extreme_w_events['StartTime(UTC)'].str.split(' ', expand=True)
w_events_date_start = w_events_date_start[0].str.split('-', expand=True)
extreme_w_events['event_start_dt'] = (w_events_date_start[0].astype(int) * 10000) + (w_events_date_start[1].astype(int) * 100) + (w_events_date_start[2].astype(int) * 1)

w_events_date_end = extreme_w_events['EndTime(UTC)'].str.split(' ', expand=True)
w_events_date_end = w_events_date_end[0].str.split('-', expand=True)
extreme_w_events['event_end_dt'] = (w_events_date_end[0].astype(int) * 10000) + (w_events_date_end[1].astype(int) * 100) + (w_events_date_end[2].astype(int) * 1)

C:\Users\Skurai\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Skurai\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [9]:
extreme_w_events.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,wcoordinateID,event_start_dt,event_end_dt
6,W-7,Fog,Severe,2016-01-09 12:54:00,2016-01-09 15:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,"(38.1, -106.17)",20160109,20160109
8,W-9,Fog,Severe,2016-01-09 16:14:00,2016-01-09 16:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,"(38.1, -106.17)",20160109,20160109
10,W-11,Cold,Severe,2016-01-09 16:54:00,2016-01-09 20:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,"(38.1, -106.17)",20160109,20160109
11,W-12,Fog,Severe,2016-01-10 02:54:00,2016-01-10 04:14:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,"(38.1, -106.17)",20160110,20160110
12,W-13,Fog,Severe,2016-01-10 09:34:00,2016-01-10 10:14:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,"(38.1, -106.17)",20160110,20160110


In [10]:
m_safety_events = pd.read_csv('Major_Safety_Events.csv')
cities = pd.read_csv('us_cities.csv')
# https://github.com/kelvins/US-Cities-Database

C:\Users\Skurai\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1,32,33,36,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
m_safety_events.columns

Index(['5 Digit NTD ID', '4 Digit NTD ID', 'Agency', 'Organization Type',
       'Mode', 'Type of Service', 'Primary UZA Code', 'Primary UZA Name',
       'Primary UZA Sq Miles', 'Primary UZA Population',
       'Service Area Sq Miles', 'Service Area Population', 'Year',
       'Incident Number', 'Incident Date', 'Incident Time', 'Event Type',
       'Other Event Type Description', 'Safety/Security', 'Collision With',
       'Property Damage', 'Total Injuries', 'Total Fatalities',
       'Life Safety (Y/N)', 'Number of Transit Vehicles Involved',
       'Towed (Y/N)', 'Number of Vehicles Involved', 'Location Type',
       'Incident Location', 'Evacuation Comment', 'Self Evacuation (Y/N)',
       'Evacuation Location', 'HazMat Type', 'HazMat Type Description',
       'Fire Type', 'Fire Fuel', 'Other Fire Fuel Description', 'Weather',
       'Lighting', 'Current Condition', 'Tide', 'Road Configuration',
       'Track Configuration', 'Path Condition', 'Rail Alignment',
       'Rail Grade 

In [12]:
m_safety_events.dropna(subset=['Primary UZA Name'],inplace=True)

In [13]:
safety_events_citystate = m_safety_events['Primary UZA Name'].str.split(',', expand=True)
safety_events_state = safety_events_citystate[1].str.split('-', expand=True)
safety_events_state[0] = safety_events_state[0].str.strip()

In [14]:
m_safety_events['City'] = safety_events_citystate[0]
m_safety_events['State'] = safety_events_state[0]

In [15]:
m_safety_events.head()

,5 Digit NTD ID,4 Digit NTD ID,Agency,Organization Type,Mode,Type of Service,Primary UZA Code,Primary UZA Name,Primary UZA Sq Miles,Primary UZA Population,...,Pedestrian or Bicyclist Injuries,Pederstiran in Crosswalk Injuries,Pedestrian Not in Crosswalk Injuries,Pedestrian Crossing Tracks Injuries,Pedestrian Walking Along Tracks Injuries,Occupant of Other Vehicle Injuries,Other Injuries,Suicide Injuries,City,State
0,50157.0,5157,Butler County Regional Transit Authority,Independent Public Agency or Authority of Tran...,CB,DO,30,"Cincinnati, OH-KY-IN",788.0,1624827.0,...,0,0,0,0,0,1,0,0,Cincinnati,OH
1,50157.0,5157,Butler County Regional Transit Authority,Independent Public Agency or Authority of Tran...,DR,DO,30,"Cincinnati, OH-KY-IN",788.0,1624827.0,...,0,0,0,0,0,0,0,0,Cincinnati,OH
2,40130.0,4130,Macon-Bibb County Transit Authority,Independent Public Agency or Authority of Tran...,MB,DO,234,"Macon, GA",98.0,137570.0,...,0,0,0,0,0,0,0,0,Macon,GA
3,20137.0,2137,Monroe Bus Corporation,Private-For-Profit Corporation,CB,DO,89,"Poughkeepsie-Newburgh, NY-NJ",327.0,423566.0,...,0,0,0,0,0,26,0,0,Poughkeepsie-Newburgh,NY
4,90014.0,9014,Alameda-Contra Costa Transit District,Independent Public Agency or Authority of Tran...,MB,DO,13,"San Francisco-Oakland, CA",524.0,3281212.0,...,0,0,0,0,0,0,0,0,San Francisco-Oakland,CA


In [16]:
m_safety_events = m_safety_events.merge(cities, how='inner', left_on=['City','State'], right_on=['CITY','STATE_CODE'])

In [17]:
m_safety_events['coordinateID'] = list(zip(round(m_safety_events['LATITUDE'],1),round(m_safety_events['LONGITUDE'],1)))
m_safety_events.head()

,5 Digit NTD ID,4 Digit NTD ID,Agency,Organization Type,Mode,Type of Service,Primary UZA Code,Primary UZA Name,Primary UZA Sq Miles,Primary UZA Population,...,City,State,ID,STATE_CODE,STATE_NAME,CITY,COUNTY,LATITUDE,LONGITUDE,coordinateID
0,50157.0,5157,Butler County Regional Transit Authority,Independent Public Agency or Authority of Tran...,CB,DO,30,"Cincinnati, OH-KY-IN",788.0,1624827.0,...,Cincinnati,OH,19689,OH,Ohio,Cincinnati,Clermont,39.091293,-84.277383,"(39.09, -84.28)"
1,50157.0,5157,Butler County Regional Transit Authority,Independent Public Agency or Authority of Tran...,CB,DO,30,"Cincinnati, OH-KY-IN",788.0,1624827.0,...,Cincinnati,OH,19690,OH,Ohio,Cincinnati,Hamilton,39.106300,-84.535600,"(39.11, -84.54)"
2,50157.0,5157,Butler County Regional Transit Authority,Independent Public Agency or Authority of Tran...,DR,DO,30,"Cincinnati, OH-KY-IN",788.0,1624827.0,...,Cincinnati,OH,19689,OH,Ohio,Cincinnati,Clermont,39.091293,-84.277383,"(39.09, -84.28)"
3,50157.0,5157,Butler County Regional Transit Authority,Independent Public Agency or Authority of Tran...,DR,DO,30,"Cincinnati, OH-KY-IN",788.0,1624827.0,...,Cincinnati,OH,19690,OH,Ohio,Cincinnati,Hamilton,39.106300,-84.535600,"(39.11, -84.54)"
4,50012.0,5012,Southwest Ohio Regional Transit Authority,"City, County or Local Government Unit or Depar...",MB,DO,30,"Cincinnati, OH-KY-IN",788.0,1624827.0,...,Cincinnati,OH,19689,OH,Ohio,Cincinnati,Clermont,39.091293,-84.277383,"(39.09, -84.28)"


In [18]:
safety_events_date = m_safety_events['Incident Date'].str.split(' ', expand=True)
safety_events_date = safety_events_date[0].str.split('/', expand=True)
safety_events_date

m_safety_events['event_date'] = (safety_events_date[2].astype(int) * 10000) + (safety_events_date[1].astype(int) * 100) + (safety_events_date[0].astype(int) * 1)

In [19]:
m_safety_events.head()

,5 Digit NTD ID,4 Digit NTD ID,Agency,Organization Type,Mode,Type of Service,Primary UZA Code,Primary UZA Name,Primary UZA Sq Miles,Primary UZA Population,...,State,ID,STATE_CODE,STATE_NAME,CITY,COUNTY,LATITUDE,LONGITUDE,coordinateID,event_date
0,50157.0,5157,Butler County Regional Transit Authority,Independent Public Agency or Authority of Tran...,CB,DO,30,"Cincinnati, OH-KY-IN",788.0,1624827.0,...,OH,19689,OH,Ohio,Cincinnati,Clermont,39.091293,-84.277383,"(39.09, -84.28)",20140502
1,50157.0,5157,Butler County Regional Transit Authority,Independent Public Agency or Authority of Tran...,CB,DO,30,"Cincinnati, OH-KY-IN",788.0,1624827.0,...,OH,19690,OH,Ohio,Cincinnati,Hamilton,39.106300,-84.535600,"(39.11, -84.54)",20140502
2,50157.0,5157,Butler County Regional Transit Authority,Independent Public Agency or Authority of Tran...,DR,DO,30,"Cincinnati, OH-KY-IN",788.0,1624827.0,...,OH,19689,OH,Ohio,Cincinnati,Clermont,39.091293,-84.277383,"(39.09, -84.28)",20141701
3,50157.0,5157,Butler County Regional Transit Authority,Independent Public Agency or Authority of Tran...,DR,DO,30,"Cincinnati, OH-KY-IN",788.0,1624827.0,...,OH,19690,OH,Ohio,Cincinnati,Hamilton,39.106300,-84.535600,"(39.11, -84.54)",20141701
4,50012.0,5012,Southwest Ohio Regional Transit Authority,"City, County or Local Government Unit or Depar...",MB,DO,30,"Cincinnati, OH-KY-IN",788.0,1624827.0,...,OH,19689,OH,Ohio,Cincinnati,Clermont,39.091293,-84.277383,"(39.09, -84.28)",20142901


In [25]:
m_safety_events.join(extreme_w_events.set_index(['City','State']), how='inner', on=['City','State'], lsuffix='safety_', rsuffix='weather_')

# https://stackoverflow.com/questions/47386405/memoryerror-when-i-merge-two-pandas-data-frames

# memory error, also, need to merge on coordinateID / wcoordinateID  + date, or city, state + date

MemoryError: 